# Importing Necessary Libraries

In [65]:
import pandas as pd
import re
from datetime import timedelta
import sys
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import datetime
from textblob import TextBlob
from afinn import Afinn
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Loading the Dataset

In [66]:
df=pd.read_excel('Lysol BW 11-17th Aug.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4270 entries, 0 to 4269
Columns: 117 entries, ID to Translated Text
dtypes: bool(5), float64(30), int64(25), object(57)
memory usage: 3.7+ MB


# Analysing the Patterns from the Dataset

In [67]:
pd.set_option('display.max_columns',None)
df.head()

ID    Query Id Query Name                   Date  \
0   1  2000458559      Lysol  2022-08-17 23:57:07.0   
1   2  2000458559      Lysol  2022-08-17 23:54:07.0   
2   3  2000458559      Lysol  2022-08-17 23:51:09.0   
3   4  2000458559      Lysol  2022-08-17 23:48:51.0   
4   5  2000458559      Lysol  2022-08-17 23:48:25.0   

                                               Title  \
0  @AmyJPetty I hope you took Lysol. There seems ...   
1  Can't locate the source of mold/mildew smell -...   
2  Me in the house with the windows closed: \n<Ki...   
3  walmart gave us free lysol wipes and a carton ...   
4  Can y’all list off some household items you re...   

                                                 Url       Domain Sentiment  \
0  http://twitter.com/csnetprogrammer/statuses/15...  twitter.com  positive   
1  https://www.reddit.com/r/fordescape/comments/w...   reddit.com  positive   
2  http://twitter.com/tracieTweetsTv/statuses/156...  twitter.com  negative   
3  http://twitter.com/yogurteater4/statuses/15600...  twitter.com   neutral   
4  https://www.reddit.com/r/aldi/comments/wqw9ef/...   reddit.com  negative   

  Page Type Language Country Code Continent Code      Continent  \
0   twitter       en          NaN            NaN            NaN   
1    reddit       en          NaN            NaN            NaN   
2   twitter       en          USA  NORTH AMERICA  North America   
3   twitter       en          NaN            NaN            NaN   
4    reddit       en          NaN            NaN            NaN   

                    Country City Code Account Type  \
0                       NaN       NaN   individual   
1                       NaN       NaN          NaN   
2  United States of America       NaN   individual   
3                       NaN       NaN   individual   
4                       NaN       NaN          NaN   

                          Added  Assignment           Author  \
0  2022-08-18T00:00:46.170+0000         NaN  csnetprogrammer   
1  2022-08-17T23:56:52.409+0000         NaN       goforaudio   
2  2022-08-17T23:56:50.294+0000         NaN   tracieTweetsTv   
3  2022-08-17T23:55:35.593+0000         NaN     yogurteater4   
4  2022-08-17T23:49:58.166+0000         NaN    MonkeyPretzel   

                                              Avatar  \
0  https://audiences.brandwatch.com/api/audiences...   
1  https://www.redditstatic.com/avatars/defaults/...   
2  https://audiences.brandwatch.com/api/audiences...   
3  https://audiences.brandwatch.com/api/audiences...   
4  https://styles.redditmedia.com/t5_17u4qw/style...   

                                    Category Details  Checked City  \
0                                                NaN    False  NaN   
1                                                NaN    False  NaN   
2                                                NaN    False  NaN   
3  {id=9631167, name=Disinfecting Wipes, displayN...    False  NaN   
4                                                NaN    False  NaN   

   Display URLs Expanded URLs  Facebook Author ID  Facebook Comments  \
0           NaN           NaN                 NaN                  0   
1           NaN           NaN                 NaN                  0   
2           NaN           NaN                 NaN                  0   
3           NaN           NaN                 NaN                  0   
4           NaN           NaN                 NaN                  0   

   Facebook Likes Facebook Role  Facebook Shares Facebook Subtype  \
0               0           NaN                0              NaN   
1               0           NaN                0              NaN   
2               0           NaN                0              NaN   
3               0           NaN                0              NaN   
4               0           NaN                0              NaN   

                          Full Name  \
0            csnetprogrammer (Gary)   
1                        goforaudio   
2         tracieTweetsTv

In [68]:
df.columns

Index(['ID', 'Query Id', 'Query Name', 'Date', 'Title', 'Url', 'Domain',
       'Sentiment', 'Page Type', 'Language',
       ...
       'Reddit Score Upvote Ratio', 'Region', 'Region Code', 'Root Blog Name',
       'Root Post Id', 'Subreddit', 'Subreddit Subscribers', 'Weblog Title',
       'Unnamed: 115', 'Translated Text'],
      dtype='object', length=117)

# Checking Sentiments

## TextBlob

In [69]:
def sentiment_calc(text):
    try:
        return TextBlob(text).sentiment.polarity
    except:
        return None


In [70]:
df['sentiment_polarity1'] = df['Translated Text'].apply(sentiment_calc)

In [71]:
sentiment_list = []
for i in df['sentiment_polarity1']:
    if i > 0:
        sentiment_list.append('Positive')
    elif i < 0:
        sentiment_list.append('Negative')
    else:
        sentiment_list.append('Neutral')
df['sentiment_t']=sentiment_list
        

In [72]:
pd.crosstab(index=df["sentiment_t"],columns="count") 

col_0        count
sentiment_t       
Negative       898
Neutral       1492
Positive      1880

## Affin

In [73]:
af = Afinn()

def sentiment_calc1(text):
    try:
        return af.score(text)
    except:
        return None


In [74]:
df['sentiment_polarity2'] = df['Translated Text'].apply(sentiment_calc1)

In [75]:
sentiment_list = []
for i in df['sentiment_polarity2']:
    if i > 0:
        sentiment_list.append('Positive')
    elif i < 0:
        sentiment_list.append('Negative')
    else:
        sentiment_list.append('Neutral')
df['sentiment_a']=sentiment_list
        

In [76]:
pd.crosstab(index=df["sentiment_a"],columns="count") 

col_0        count
sentiment_a       
Negative      1069
Neutral       1398
Positive      1803

## Vander Sentiment

In [77]:
sid_obj = SentimentIntensityAnalyzer()

def sentiment_scores(sentence):
    try:
        return sid_obj.polarity_scores(sentence) 
    except:
        return None

In [78]:
sentiment_dict = df['Translated Text'].apply(sentiment_scores)

In [79]:
df["senti_dict"]=sentiment_dict

In [80]:
sentiment_dict[0]["compound"]

0.296

In [81]:
a=[]

for i in range(len(df["senti_dict"])):
    try:
        a.append(df["senti_dict"][i]['compound'])
    except:
        a.append("NA")


In [82]:
df["van_score"]=a

In [83]:
df.head()

ID    Query Id Query Name                   Date  \
0   1  2000458559      Lysol  2022-08-17 23:57:07.0   
1   2  2000458559      Lysol  2022-08-17 23:54:07.0   
2   3  2000458559      Lysol  2022-08-17 23:51:09.0   
3   4  2000458559      Lysol  2022-08-17 23:48:51.0   
4   5  2000458559      Lysol  2022-08-17 23:48:25.0   

                                               Title  \
0  @AmyJPetty I hope you took Lysol. There seems ...   
1  Can't locate the source of mold/mildew smell -...   
2  Me in the house with the windows closed: \n<Ki...   
3  walmart gave us free lysol wipes and a carton ...   
4  Can y’all list off some household items you re...   

                                                 Url       Domain Sentiment  \
0  http://twitter.com/csnetprogrammer/statuses/15...  twitter.com  positive   
1  https://www.reddit.com/r/fordescape/comments/w...   reddit.com  positive   
2  http://twitter.com/tracieTweetsTv/statuses/156...  twitter.com  negative   
3  http://twitter.com/yogurteater4/statuses/15600...  twitter.com   neutral   
4  https://www.reddit.com/r/aldi/comments/wqw9ef/...   reddit.com  negative   

  Page Type Language Country Code Continent Code      Continent  \
0   twitter       en          NaN            NaN            NaN   
1    reddit       en          NaN            NaN            NaN   
2   twitter       en          USA  NORTH AMERICA  North America   
3   twitter       en          NaN            NaN            NaN   
4    reddit       en          NaN            NaN            NaN   

                    Country City Code Account Type  \
0                       NaN       NaN   individual   
1                       NaN       NaN          NaN   
2  United States of America       NaN   individual   
3                       NaN       NaN   individual   
4                       NaN       NaN          NaN   

                          Added  Assignment           Author  \
0  2022-08-18T00:00:46.170+0000         NaN  csnetprogrammer   
1  2022-08-17T23:56:52.409+0000         NaN       goforaudio   
2  2022-08-17T23:56:50.294+0000         NaN   tracieTweetsTv   
3  2022-08-17T23:55:35.593+0000         NaN     yogurteater4   
4  2022-08-17T23:49:58.166+0000         NaN    MonkeyPretzel   

                                              Avatar  \
0  https://audiences.brandwatch.com/api/audiences...   
1  https://www.redditstatic.com/avatars/defaults/...   
2  https://audiences.brandwatch.com/api/audiences...   
3  https://audiences.brandwatch.com/api/audiences...   
4  https://styles.redditmedia.com/t5_17u4qw/style...   

                                    Category Details  Checked City  \
0                                                NaN    False  NaN   
1                                                NaN    False  NaN   
2                                                NaN    False  NaN   
3  {id=9631167, name=Disinfecting Wipes, displayN...    False  NaN   
4                                                NaN    False  NaN   

   Display URLs Expanded URLs  Facebook Author ID  Facebook Comments  \
0           NaN           NaN                 NaN                  0   
1           NaN           NaN                 NaN                  0   
2           NaN           NaN                 NaN                  0   
3           NaN           NaN                 NaN                  0   
4           NaN           NaN                 NaN                  0   

   Facebook Likes Facebook Role  Facebook Shares Facebook Subtype  \
0               0           NaN                0              NaN   
1               0           NaN                0              NaN   
2               0           NaN                0              NaN   
3               0           NaN                0              NaN   
4               0           NaN                0              NaN   

                          Full Name  \
0            csnetprogrammer (Gary)   
1                        goforaudio   
2         tracieTweetsTv

In [84]:
s_list=[]


for i in df["van_score"]:
    try:
        if i > 0.1:
            s_list.append('Positive')
        elif i < 0:
            s_list.append('Negative')
        else:
            s_list.append('Neutral')
    except:
            s_list.append('NA')

df['sentiment1']=s_list        

In [85]:
pd.crosstab(index=df["sentiment1"],columns="count") 


col_0       count
sentiment1       
NA             29
Negative     1037
Neutral      1064
Positive     2140

In [86]:
# Merging all the columns into a single column

df['Analytics Sentiment']=df[['sentiment1','sentiment_a', 'sentiment_t']].mode(axis=1)[0]

pd.crosstab(index=df['Analytics Sentiment'],columns="count") 

col_0                count
Analytics Sentiment       
Negative              1172
Neutral               1205
Positive              1893

In [87]:
# Checking columns
pd.set_option('display.max_columns',None)
df.head()

ID    Query Id Query Name                   Date  \
0   1  2000458559      Lysol  2022-08-17 23:57:07.0   
1   2  2000458559      Lysol  2022-08-17 23:54:07.0   
2   3  2000458559      Lysol  2022-08-17 23:51:09.0   
3   4  2000458559      Lysol  2022-08-17 23:48:51.0   
4   5  2000458559      Lysol  2022-08-17 23:48:25.0   

                                               Title  \
0  @AmyJPetty I hope you took Lysol. There seems ...   
1  Can't locate the source of mold/mildew smell -...   
2  Me in the house with the windows closed: \n<Ki...   
3  walmart gave us free lysol wipes and a carton ...   
4  Can y’all list off some household items you re...   

                                                 Url       Domain Sentiment  \
0  http://twitter.com/csnetprogrammer/statuses/15...  twitter.com  positive   
1  https://www.reddit.com/r/fordescape/comments/w...   reddit.com  positive   
2  http://twitter.com/tracieTweetsTv/statuses/156...  twitter.com  negative   
3  http://twitter.com/yogurteater4/statuses/15600...  twitter.com   neutral   
4  https://www.reddit.com/r/aldi/comments/wqw9ef/...   reddit.com  negative   

  Page Type Language Country Code Continent Code      Continent  \
0   twitter       en          NaN            NaN            NaN   
1    reddit       en          NaN            NaN            NaN   
2   twitter       en          USA  NORTH AMERICA  North America   
3   twitter       en          NaN            NaN            NaN   
4    reddit       en          NaN            NaN            NaN   

                    Country City Code Account Type  \
0                       NaN       NaN   individual   
1                       NaN       NaN          NaN   
2  United States of America       NaN   individual   
3                       NaN       NaN   individual   
4                       NaN       NaN          NaN   

                          Added  Assignment           Author  \
0  2022-08-18T00:00:46.170+0000         NaN  csnetprogrammer   
1  2022-08-17T23:56:52.409+0000         NaN       goforaudio   
2  2022-08-17T23:56:50.294+0000         NaN   tracieTweetsTv   
3  2022-08-17T23:55:35.593+0000         NaN     yogurteater4   
4  2022-08-17T23:49:58.166+0000         NaN    MonkeyPretzel   

                                              Avatar  \
0  https://audiences.brandwatch.com/api/audiences...   
1  https://www.redditstatic.com/avatars/defaults/...   
2  https://audiences.brandwatch.com/api/audiences...   
3  https://audiences.brandwatch.com/api/audiences...   
4  https://styles.redditmedia.com/t5_17u4qw/style...   

                                    Category Details  Checked City  \
0                                                NaN    False  NaN   
1                                                NaN    False  NaN   
2                                                NaN    False  NaN   
3  {id=9631167, name=Disinfecting Wipes, displayN...    False  NaN   
4                                                NaN    False  NaN   

   Display URLs Expanded URLs  Facebook Author ID  Facebook Comments  \
0           NaN           NaN                 NaN                  0   
1           NaN           NaN                 NaN                  0   
2           NaN           NaN                 NaN                  0   
3           NaN           NaN                 NaN                  0   
4           NaN           NaN                 NaN                  0   

   Facebook Likes Facebook Role  Facebook Shares Facebook Subtype  \
0               0           NaN                0              NaN   
1               0           NaN                0              NaN   
2               0           NaN                0              NaN   
3               0           NaN                0              NaN   
4               0           NaN                0              NaN   

                          Full Name  \
0            csnetprogrammer (Gary)   
1                        goforaudio   
2         tracieTweetsTv

In [88]:
a=df.columns
a

Index(['ID', 'Query Id', 'Query Name', 'Date', 'Title', 'Url', 'Domain',
       'Sentiment', 'Page Type', 'Language',
       ...
       'Unnamed: 115', 'Translated Text', 'sentiment_polarity1', 'sentiment_t',
       'sentiment_polarity2', 'sentiment_a', 'senti_dict', 'van_score',
       'sentiment1', 'Analytics Sentiment'],
      dtype='object', length=125)

In [89]:
# Dropping Unwanted Columns

df1=df.drop(['sentiment_polarity1', 'sentiment_t',
       'sentiment_polarity2', 'sentiment_a', 'senti_dict', 'van_score',
       'sentiment1'], axis = 1) 

In [90]:
# saving into Excel

writer = pd.ExcelWriter(r"sentiment_Lysol BW 11-17th Aug.xlsx", engine='xlsxwriter',options={'strings_to_urls': False})
df1.to_excel(writer)
writer.close()

C:\Users\ujjwal.pratik1\AppData\Local\Temp\ipykernel_18860\1605500458.py:3: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.
  writer = pd.ExcelWriter(r"sentiment_Lysol BW 11-17th Aug.xlsx", engine='xlsxwriter',options={'strings_to_urls': False})


In [91]:
df1.head(10)

ID    Query Id Query Name                   Date  \
0   1  2000458559      Lysol  2022-08-17 23:57:07.0   
1   2  2000458559      Lysol  2022-08-17 23:54:07.0   
2   3  2000458559      Lysol  2022-08-17 23:51:09.0   
3   4  2000458559      Lysol  2022-08-17 23:48:51.0   
4   5  2000458559      Lysol  2022-08-17 23:48:25.0   
5   6  2000458559      Lysol  2022-08-17 23:47:55.0   
6   7  2000458559      Lysol  2022-08-17 23:45:00.0   
7   8  2000458559      Lysol  2022-08-17 23:44:50.0   
8   9  2000458559      Lysol  2022-08-17 23:39:41.0   
9  10  2000458559      Lysol  2022-08-17 23:34:43.0   

                                               Title  \
0  @AmyJPetty I hope you took Lysol. There seems ...   
1  Can't locate the source of mold/mildew smell -...   
2  Me in the house with the windows closed: \n<Ki...   
3  walmart gave us free lysol wipes and a carton ...   
4  Can y’all list off some household items you re...   
5  @dttpeople @Jimisnyder88 @YouWannaDoWhat @JoJo...   
6  @baileyjlimon i don’t take a pic but i am trul...   
7  At what point do you just toss your clothes in...   
8  @Mima1_20 @OrlandoJNegron2 acabo de ver una li...   
9                            Free Welcome Back Packs   

                                                 Url        Domain Sentiment  \
0  http://twitter.com/csnetprogrammer/statuses/15...   twitter.com  positive   
1  https://www.reddit.com/r/fordescape/comments/w...    reddit.com  positive   
2  http://twitter.com/tracieTweetsTv/statuses/156...   twitter.com  negative   
3  http://twitter.com/yogurteater4/statuses/15600...   twitter.com   neutral   
4  https://www.reddit.com/r/aldi/comments/wqw9ef/...    reddit.com  negative   
5  http://twitter.com/amesj/statuses/156005082504...   twitter.com   neutral   
6  http://twitter.com/baileyjlimon/statuses/15600...   twitter.com  negative   
7  https://www.reddit.com/r/running/comments/wr39...    reddit.com   neutral   
8  http://twitter.com/AnnettePRico/statuses/15600...   twitter.com   neutral   
9  https://www.facebook.com/144407492257288/posts...  facebook.com   neutral   

  Page Type Language Country Code Continent Code      Continent  \
0   twitter       en          NaN            NaN            NaN   
1    reddit       en          NaN            NaN            NaN   
2   twitter       en          USA  NORTH AMERICA  North America   
3   twitter       en          NaN            NaN            NaN   
4    reddit       en          NaN            NaN            NaN   
5   twitter       en          NaN            NaN            NaN   
6   twitter       en          USA  NORTH AMERICA  North America   
7    reddit       en          NaN            NaN            NaN   
8   twitter       es          PRI  NORTH AMERICA  North America   
9  facebook       en          NaN            NaN            NaN   

                    Country                 City Code Account Type  \
0                       NaN                       NaN   individual   
1                       NaN                       NaN          NaN   
2  United States of America                       NaN   individual   
3                       NaN                       NaN   individual   
4                       NaN                       NaN          NaN   
5                       NaN                       NaN   individual   
6  United States of America                       NaN   individual   
7                       NaN                       NaN          NaN   
8               Puerto Rico  PRI.Puerto Rico.San Juan   individual   
9                       NaN                       NaN          NaN   

                          Added  Assignment           Author  \
0  2022-08-18T00:00:46.170+0000         NaN  csnetprogrammer   
1  2022-08-17T23:56:52.409+0000         NaN       goforaudio   
2  2022-08-17T23:56:50.294+0000         NaN   tracieTweetsTv   
3  2022-08-17T23:55:35.593+0000         NaN     yogurteater4   
4  2022-08-17T23:49:58.166+0000         NaN    MonkeyPretzel   
5  202